# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Segmentos

In [65]:
query = '''
SELECT
    DISTINCT
    SO.OFERTATA,
    SO.FECHA,
    SO.PERFIL_CLIENTE,
    SO.SOCI_SOCI_ID,
    SO.SUBCLASE,
    SUB.SUB_NAME
FROM
    SANDBOX_PLUS.DWH.SEGMENTOS_OFERTATA AS SO
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = SO.SUBCLASE
WHERE
    SO.OFERTATA ILIKE '%7%'
'''

cursor.execute(query)
df = cursor.fetch_pandas_all()
df.head(2)

,OFERTATA,FECHA,PERFIL_CLIENTE,SOCI_SOCI_ID,SUBCLASE,SUB_NAME
0,Ofertata 7,2024-02-15,ABANDONADOR,902735,1700020001,AL HUEVO
1,Ofertata 7,2024-02-15,ABANDONADOR,1301700,1700020001,AL HUEVO


In [22]:
fecha_ofertata = df.loc[0, 'FECHA']
fecha_ofertata

datetime.date(2024, 2, 15)

In [44]:
ofertata = df.loc[0, 'OFERTATA']
ofertata

'Ofertata 7'

In [23]:
df['PERFIL_CLIENTE'].value_counts()

ACTIVO         157804
ABANDONADOR    144777
Name: PERFIL_CLIENTE, dtype: int64

# Abandonadores

In [24]:
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [34]:
abandonador = df[df['PERFIL_CLIENTE'] == 'ABANDONADOR']
abandonador['K'] = abandonador['SOCI_SOCI_ID'].astype(str) + abandonador['SUBCLASE'].astype(str)
abandonador.head(2)

,FECHA,PERFIL_CLIENTE,SOCI_SOCI_ID,SUBCLASE,K
0,2024-02-15,ABANDONADOR,902735,1700020001,9027351700020001
1,2024-02-15,ABANDONADOR,451573,1700020001,4515731700020001


## Impactados

### A. Vinieron a TATA

In [32]:
query = '''
SELECT
    DISTINCT FFM.SOCI_SOCI_ID
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
WHERE
    FFM.TIEM_DIA_ID BETWEEN '{fecha_snow}' AND DATEADD(DAY, 7, '{fecha_snow}')
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
'''

cursor.execute(query.format(fecha_snow = fecha_ofertata))
volvieron_general = cursor.fetch_pandas_all()
volvieron_general = volvieron_general[volvieron_general['SOCI_SOCI_ID'].isin(abandonador['SOCI_SOCI_ID'])]
print(volvieron_general['SOCI_SOCI_ID'].nunique())
volvieron_general.head(2)

11855


,SOCI_SOCI_ID
7,1902906
13,50886


### B. Vinieron en 2024 y consumieron alguna de las SUBCLASE

In [33]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    SUB.SUBCLASE IN {subclases_snow}
    AND FFM.TIEM_DIA_ID BETWEEN '{fecha_snow}' AND DATEADD(DAY, 7, '{fecha_snow}')
'''

cursor.execute(query.format(
    subclases_snow = tuple(abandonador['SUBCLASE'].unique()),
    fecha_snow = fecha_ofertata
    ))
volvieron_subclase = cursor.fetch_pandas_all()

volvieron_subclase.head(2)

,TICKET,SOCI_SOCI_ID,SUBCLASE
0,202402152342235474,190674,1700020001
1,202402152351136490,1788622,1700020001


In [35]:
print(volvieron_subclase.shape[0])
volvieron_subclase_2 = volvieron_subclase[(volvieron_subclase['SOCI_SOCI_ID'].isin(abandonador['SOCI_SOCI_ID'])) & (volvieron_subclase['SUBCLASE'].isin(abandonador['SUBCLASE']))]
print(volvieron_subclase_2.shape[0])
print(volvieron_subclase_2['SOCI_SOCI_ID'].nunique())

96553
3951
2221


In [97]:
# Compraron
resumen_vinieron_2024_2 = volvieron_subclase_2.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
resumen_vinieron_2024_2.rename({'SOCI_SOCI_ID':'COMPRARON'}, axis = 1, inplace = True)

# Impactados
impactados_segm_5 = abandonador.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
impactados_segm_5.rename({'SOCI_SOCI_ID':'IMPACTADOS'}, axis = 1, inplace = True)

# Consolido
resumen_vinieron_2024_2 = resumen_vinieron_2024_2.merge(impactados_segm_5, on = 'SUBCLASE', how = 'inner')
resumen_vinieron_2024_2['RATIO'] = (resumen_vinieron_2024_2['COMPRARON'] / resumen_vinieron_2024_2['IMPACTADOS'] * 100).astype(int)

resumen_vinieron_2024_2 = resumen_vinieron_2024_2.merge(df[['SUB_NAME', 'SUBCLASE']].drop_duplicates(), on = 'SUBCLASE', how = 'inner')

columns = resumen_vinieron_2024_2.columns.tolist()
new_columns = [columns[-1]] + columns[:-1]
resumen_vinieron_2024_2 = resumen_vinieron_2024_2[new_columns]

resumen_vinieron_2024_2.sort_values(by = 'RATIO', ascending = False, inplace = True)
resumen_vinieron_2024_2

,SUB_NAME,SUBCLASE,COMPRARON,IMPACTADOS,RATIO
1,SOJA,1100010006,484,21779,2
2,MAYONESA TRADICIONAL,1200010004,452,21656,2
3,AL HUEVO,1700020001,486,24051,2
4,JAMON COCIDO AL CORTE,4000030002,586,26147,2
0,BONDIOLA,1000030001,373,25553,1
5,LIMPIADORES LIQUIDOS MULTISUPERFICIES,8900060002,451,25591,1


### Conclusiones Impactados

In [100]:
a = abandonador['SOCI_SOCI_ID'].nunique()
b = int(volvieron_general['SOCI_SOCI_ID'].nunique() / abandonador['SOCI_SOCI_ID'].nunique() * 100)
c = int(volvieron_subclase_2['SOCI_SOCI_ID'].nunique() / abandonador['SOCI_SOCI_ID'].nunique() * 100)
d = str(resumen_vinieron_2024_2.loc[:3, 'SUB_NAME'].values).replace('[', '').replace(']', '')

print(f"Los segmentos de {ofertata} fueron generados el {fecha_ofertata}")
print("")
print("---- Abandonadores")
print("")
print(f"Comprende clientes que en los ultimos 30 dias no vengan a TATA y tengan Score 4 o 5 en la subclase")
print(f"De los {a} clientes que cumplen dicho criterio, un {b}% volvieron a TATA luego del {ofertata}")
print("")
print(f"Asimismo, de los {a} clientes, un {c}% volvieron a TATA luego del {ofertata} y compraron la subclase que se les promociono, siendo {d} las mejores subclases")

Los segmentos de Ofertata 7 fueron generados el 2024-02-15

---- Abandonadores

Comprende clientes que en los ultimos 30 dias no vengan a TATA y tengan Score 4 o 5 en la subclase
De los 144777 clientes que cumplen dicho criterio, un 8% volvieron a TATA luego del Ofertata 7

Asimismo, de los 144777 clientes, un 1% volvieron a TATA luego del Ofertata 7 y compraron la subclase que se les promociono, siendo 'SOJA' 'MAYONESA TRADICIONAL' 'AL HUEVO' las mejores subclases


## Venta y GB1

In [57]:
query = '''
SELECT
    FFM.SOCI_SOCI_ID,
    FFM.SOCI_SOCI_ID || SUB.SUBCLASE AS K,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    FFM.TIEM_DIA_ID BETWEEN '{fecha_snow}' AND DATEADD(DAY, 7, '{fecha_snow}')
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUBCLASE IN {subclases_snow}
GROUP BY
    ALL
'''

cursor.execute(query.format(
    subclases_snow = tuple(abandonador['SUBCLASE'].unique()),
    fecha_snow = fecha_ofertata
    ))
ventas_segm_5 = cursor.fetch_pandas_all()

ventas_segm_5 = ventas_segm_5[ventas_segm_5['K'].isin(abandonador['K'])]

print(f"Ventas en Miles: {int(round(ventas_segm_5['VENTAS'].sum() / 1000, 0))}")
print(f"GB1 en Miles: {int(round(ventas_segm_5['GB1'].sum() / 1000, 0))}")

Ventas en Miles: 84
GB1 en Miles: 16


# Activos

In [89]:
# Para cada categoria (subclase) son los clientes activos que no compran la subclase y tienen score 5

In [90]:
activo = df[df['PERFIL_CLIENTE'] == 'ACTIVO']
activo['K'] = activo['SOCI_SOCI_ID'].astype(str) + activo['SUBCLASE'].astype(str)

In [91]:
activo['SUBCLASE'].unique()

array([1700020001, 1000030001, 4000030002, 8900060002, 1200010004,
       1100010006], dtype=int64)

In [92]:
query = '''
SELECT
    FFM.TICKET,
    FFM.SOCI_SOCI_ID,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    FFM.TIEM_DIA_ID BETWEEN '{fecha_snow}' AND DATEADD(DAY, 7, '{fecha_snow}')
    AND SUB.SUBCLASE IN {subclases_snow}
'''

cursor.execute(query.format(
    subclases_snow = tuple(abandonador['SUBCLASE'].unique()),
    fecha_snow = fecha_ofertata
))
volvieron_subclase_segm_3 = cursor.fetch_pandas_all()

volvieron_subclase_segm_3.head(2)

,TICKET,SOCI_SOCI_ID,SUBCLASE
0,202402169583354439,407266,1200010004
1,202402169583354439,407266,8900060002


In [93]:
print(volvieron_subclase_segm_3.shape[0])
volvieron_subclase_segm_3_version_2 = volvieron_subclase_segm_3[(volvieron_subclase_segm_3['SOCI_SOCI_ID'].isin(activo['SOCI_SOCI_ID'])) & (volvieron_subclase_segm_3['SUBCLASE'].isin(activo['SUBCLASE']))]
print(volvieron_subclase_segm_3_version_2.shape[0])
print(volvieron_subclase_segm_3_version_2['SOCI_SOCI_ID'].nunique())

96553
49747
27632


In [101]:
# Compraron
volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
volvieron_subclase_segm_3_version_2_resumen.rename({'SOCI_SOCI_ID':'COMPRARON'}, axis = 1, inplace = True)

# Impactados
impactados_segm_3 = activo.groupby('SUBCLASE')['SOCI_SOCI_ID'].nunique().reset_index()
impactados_segm_3.rename({'SOCI_SOCI_ID':'IMPACTADOS'}, axis = 1, inplace = True)

# Consolido
volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2_resumen.merge(impactados_segm_3, on = 'SUBCLASE', how = 'inner')
volvieron_subclase_segm_3_version_2_resumen['RATIO'] = (volvieron_subclase_segm_3_version_2_resumen['COMPRARON'] / volvieron_subclase_segm_3_version_2_resumen['IMPACTADOS'] * 100).astype(int)

volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2_resumen.merge(df[['SUB_NAME', 'SUBCLASE']].drop_duplicates(), on = 'SUBCLASE', how = 'inner')

columns = volvieron_subclase_segm_3_version_2_resumen.columns.tolist()
new_columns = [columns[-1]] + columns[:-1]
volvieron_subclase_segm_3_version_2_resumen = volvieron_subclase_segm_3_version_2_resumen[new_columns]

volvieron_subclase_segm_3_version_2_resumen.sort_values(by = 'RATIO', ascending = False, inplace = True)
volvieron_subclase_segm_3_version_2_resumen

,SUB_NAME,SUBCLASE,COMPRARON,IMPACTADOS,RATIO
4,JAMON COCIDO AL CORTE,4000030002,9165,27662,33
2,MAYONESA TRADICIONAL,1200010004,5875,20091,29
3,AL HUEVO,1700020001,6649,23654,28
5,LIMPIADORES LIQUIDOS MULTISUPERFICIES,8900060002,5659,25114,22
1,SOJA,1100010006,5178,27511,18
0,BONDIOLA,1000030001,4067,33772,12


In [103]:
a = activo['SOCI_SOCI_ID'].nunique()
b = int(volvieron_subclase_segm_3_version_2['SOCI_SOCI_ID'].nunique() / activo['SOCI_SOCI_ID'].nunique() * 100)
c = str(volvieron_subclase_segm_3_version_2_resumen.loc[:3, 'SUB_NAME'].values).replace('[', '').replace(']', '')

print("---- Activos")
print("")
print(f"Comprende a los clientes activos que no compran la subclase y tienen score 5 en la misma")
print("")
print(f"De los {a} clientes que cumplen dicho criterio, un {b}% volvieron a TATA luego del {ofertata} y compraron la subclase que se les promociono, siendo {c} las mejores subclases")

---- Activos

Comprende a los clientes activos que no compran la subclase y tienen score 5 en la misma

De los 157804 clientes que cumplen dicho criterio, un 17% volvieron a TATA luego del Ofertata 7 y compraron la subclase que se les promociono, siendo 'JAMON COCIDO AL CORTE' 'MAYONESA TRADICIONAL' 'AL HUEVO' las mejores subclases


## Venta y GB1

In [109]:
query = '''
SELECT
    FFM.SOCI_SOCI_ID,
    FFM.SOCI_SOCI_ID || SUB.SUBCLASE AS K,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND) AS GB1
FROM
    MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    FFM.TIEM_DIA_ID BETWEEN '{fecha_snow}' AND DATEADD(DAY, 7, '{fecha_snow}')
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND SUB.SUBCLASE IN {subclases_snow}
GROUP BY
    ALL
'''

cursor.execute(query.format(
    subclases_snow = tuple(abandonador['SUBCLASE'].unique()),
    fecha_snow = fecha_ofertata
    ))
ventas_segm_3 = cursor.fetch_pandas_all()

ventas_segm_3 = ventas_segm_3[ventas_segm_3['K'].isin(activo['K'])]

print(f"Ventas en Miles: {int(round(ventas_segm_3['VENTAS'].sum() / 1000, 0))}")
print(f"GB1 en Miles: {int(round(ventas_segm_3['GB1'].sum() / 1000, 0))}")

Ventas en Miles: 347
GB1 en Miles: 56


# Fin